# Coset Characters

This is the development notebook for the library that calculates coset characters and partition functions for $\hat{\text{su}}(2)_k$ WZW models and their cosets

## Characters for $\hat{\text{su}}(2)_k$

We know from amazing Di-Francesco that the coset characters of $\hat{\text{su}}(2)_k$ are given by
$$
\chi_{\lambda}^{(k)} = \frac{\Theta^{(k+2)}_{\lambda + 1} - \Theta^{(k+2)}_{-\lambda - 1}}{\Theta^{(2)}_{1} - \Theta^{(2)}_{-1}}.
$$
Therefore in some way we need to calculate these $\Theta$ functions. Luckily he has also done this for us, so in the $q$ expansion we have that
$$
\chi_{\lambda}^{(k)}(q) = q^{\frac{(\lambda+1)^2}{4(k+2)} - \frac{1}{8}} \frac{\sum_{n\in \mathbb{Z}} [\lambda + 1 + 2n(k+2)] q^{n[\lambda + 1 + n(k+2)]}}{\sum_{n \in \mathbb{Z}} (1+4n)q^{n(1+2n)}}
$$
The first sum is some fractional power that contains the central charge term, but the second is not! The second has integer coefficients! The interesting part is calculating this ratio of sums.

We can quickly show that both of the polynomials are of the form
$$
a(q) = \sum_{n\in \mathbb{N}} \hat a(n) q^{\hat f(n)}, 
$$ 
where $\hat f(2n + 1) \coloneqq f(n)$, $\hat f(2n) \coloneqq f(-n)$, and similarly for $\hat a$ where $f(n) = n[\lambda + 1 + n(k+2)]$ is the exponent function over the integers. Not only that, but $\hat f$ is strictly increasing which has some nice properties.

For our numerical evaluation, we effectively have something perfect. We can store everything in a sparse array, and do operations there.

In [ ]:
from sage.all import ZZ, QQ, PowerSeriesRing, latex,O     # Relevant sage objects
from typing import Callable                 # For Typing things


# Claculate the hats
def get_character_int_coefficient_functions(lam:int, k:int):
    funcs = [ 
        lambda n: n*(lam + 1 +n*(k+2)),     # Numerator power
        lambda n: (lam + 1 + 2*n*(k+2)),    # Numerator coefficient
        lambda n: n*(1+2*n),                # Denominator power
        lambda n: 1+4*n                     # Denominator coefficient
    ]

    # Returns positive for even and negative for odd
    def get_hat(F:Callable):
        return lambda n: F([-(n-1)//2,n//2][n&1])
    
    return [get_hat(f) for f in funcs]

# Obtain some cool things
[f,a,g,b] = get_character_int_coefficient_functions(lam = 1, k = 1)


# Let's build some polynomials now
N:int = 10000                             # Number of terms
R = PowerSeriesRing(QQ,'q', sparse=True, default_prec=N)  # Our polynomial ring over the integers
q = R.gen()

numerator_coefficients   = { f(n): a(n) for n in range(1,N) }
denominator_coefficients = { g(n): b(n) for n in range(1,N) }

numerator   = R(numerator_coefficients)
denominator = R(denominator_coefficients)


In [ ]:
inv = denominator.inverse_mod(q**1000)

51*q^4 + 22*q^3 + 9*q^2 + 3*q + 1

In [ ]:
# # Define a little printing function for a polynomial up to order n
# from IPython.display import display, Latex

# def show_n(f,n:float=10000, show:bool=True):
#     terms = f.to_dict()
#     monomials = []
#     for power in reversed(terms):
#         if power[0] > n: 
#             break
#         monomials.append(r'%d q^{%d}'%(terms[power], power[0]))

    
#     string = " +".join(monomials)\
#         .replace("+-","-")\
#         .replace(r" q^{0}","")
    
#     if not show:
#         return string
    
#     display(Latex("$"+string+"$"))

# show_n(f)

<IPython.core.display.Latex object>